In [1]:
# Cell 1: Import Dependencies

from flask import Flask, jsonify
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import numpy as np

In [3]:
# Cell 2: Database Setup

# Create engine to connect to the SQLite database
database_path = "hawaii.sqlite"
engine = create_engine(f"sqlite:///D:/Promi/SurfsUp/Starter_Code/Resources/hawaii.sqlite")

# Reflect the database schema into classes
Base = automap_base()
Base.prepare(autoload_with=engine)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create a session to link Python to the database
session = Session(engine)

In [4]:
# Cell 3: Flask Setup

# Create the Flask app
app = Flask(__name__)

In [5]:
# Cell 4: Flask Routes - Home Route

@app.route("/")
def welcome():
    """List all available API routes."""
    return (
        f"Welcome to the Climate App API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start<br/>"
        f"/api/v1.0/start/end<br/>"
    )

In [6]:
# Cell 5: Flask Routes - Precipitation Data

@app.route("/api/v1.0/precipitation")
def precipitation():
    """Return a JSON list of precipitation data for the last year."""
    # Find the most recent date
    recent_date = session.query(func.max(Measurement.date)).scalar()
    one_year_ago = datetime.strptime(recent_date, '%Y-%m-%d') - timedelta(days=365)

    # Query precipitation data
    precipitation_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()

    # Convert the query results to a dictionary
    precipitation_dict = {date: prcp for date, prcp in precipitation_data}
    return jsonify(precipitation_dict)

In [7]:
# Cell 6: Flask Routes - Station Data

@app.route("/api/v1.0/stations")
def stations():
    """Return a JSON list of stations."""
    # Query all stations
    stations = session.query(Station.station).all()

    # Unpack the data and jsonify
    station_list = [station[0] for station in stations]
    return jsonify(station_list)

In [8]:
# Cell 7: Flask Routes - Temperature Observations (TOBS)

@app.route("/api/v1.0/tobs")
def tobs():
    """Return a JSON list of temperature observations for the last year."""
    # Find the most recent date
    recent_date = session.query(func.max(Measurement.date)).scalar()
    one_year_ago = datetime.strptime(recent_date, '%Y-%m-%d') - timedelta(days=365)

    # Query temperature observations for the last year
    tobs_data = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date >= one_year_ago).all()

    # Return the results as a JSON
    tobs_list = [{date: temp} for date, temp in tobs_data]
    return jsonify(tobs_list)

In [9]:
# Cell 8: Flask Routes - Start Date Route

@app.route("/api/v1.0/<start>")
def start(start):
    """Return a JSON list of the min, avg, and max temperature for all dates >= start."""
    results = session.query(
        func.min(Measurement.tobs),
        func.avg(Measurement.tobs),
        func.max(Measurement.tobs)
    ).filter(Measurement.date >= start).all()

    # Convert the results to a dictionary
    temperature_stats = {
        "Start Date": start,
        "TMIN": results[0][0],
        "TAVG": results[0][1],
        "TMAX": results[0][2]
    }
    return jsonify(temperature_stats)

In [10]:
# Cell 9: Flask Routes - Start/End Date Route

@app.route("/api/v1.0/<start>/<end>")
def start_end(start, end):
    """Return a JSON list of the min, avg, and max temperature for a date range."""
    results = session.query(
        func.min(Measurement.tobs),
        func.avg(Measurement.tobs),
        func.max(Measurement.tobs)
    ).filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    # Convert the results to a dictionary
    temperature_stats = {
        "Start Date": start,
        "End Date": end,
        "TMIN": results[0][0],
        "TAVG": results[0][1],
        "TMAX": results[0][2]
    }
    return jsonify(temperature_stats)

In [11]:
# Cell 10: Run the Flask App

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\nafis\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
